**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Sep 10 15:10:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'160205020 (1).pdf'
'160205020 (2).pdf'
'160205020 (3).pdf'
'160205020 (4).pdf'
'160205020 (5).pdf'
'160205020 (6).pdf'
'160205020 (7).pdf'
'160205020 (8).pdf'
'160205020 (9).pdf'
 16.02.05.020_C_ComTheory_Assignment_1,2.pdf
 160205020_EEE4134-Exam-1.pdf
 160205020_EEE4134-Exam2.pdf
'160205020 EEE4155.pdf'
 160205020_EEE4237_ST1.pdf
 160205020.pdf
 160205020_PPE_Assessment.pdf
 160205020_vlsi_exam3.pdf
 Classroom
'Colab Notebooks'
 covxnet
'EEE3203_160205020_PartA (1).pdf'
 EEE3203_160205020_PartA.pdf
 EEE3203_160205020_PartB.pdf
 EEE3205_160205020_PartA.pdf
 EEE3205_160205020_PartB.pdf
 EEE3205_160205020_TopPage.pdf
'EEE3207_160205020_PartA (1).pdf'
 EEE3207_160205020_PartADemo.pdf
 EEE3207_160205020_PartA.pdf
 EEE3207_160205020_PartB.pdf
 EEE3209_160205020_PartA.pdf
 EEE3209_160205020_PartB.pdf
 EEE3217_160205020_PartA.pdf
 EEE3217_160205020_PartB.pdf
'EEE4105_160205020_PartA (1).pdf'
'EEE4105_160205020_PartA (2).pdf'
 EEE4105_160205020_PartA.pdf
 EEE4105_1

**1) Clone, configure & compile Darknet**



In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15308, done.
remote: Total 15308 (delta 0), reused 0 (delta 0), pack-reused 15308
Receiving objects: 100% (15308/15308), 13.69 MiB | 17.77 MiB/s, done.
Resolving deltas: 100% (10404/10404), done.


In [ ]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# Compile
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

**2) Configure yolov3.cfg file**

In [ ]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 2000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

**3) Create .names and .data files**

In [ ]:
!echo -e 'Rickshaw' > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

**4) Save yolov3_training.cfg and obj.names files in Google drive**

In [ ]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

**5) Create a folder and unzip image dataset**

In [ ]:
!mkdir data/obj
!unzip /mydrive/yolov3/Images.zip -d data/obj

Archive:  /mydrive/yolov3/Images.zip
  inflating: data/obj/Images/ahasan0.jpg  
  inflating: data/obj/Images/ahasan0.txt  
  inflating: data/obj/Images/ahasan1.jpg  
  inflating: data/obj/Images/ahasan1.txt  
  inflating: data/obj/Images/ahasan1000.jpg  
  inflating: data/obj/Images/ahasan1000.txt  
  inflating: data/obj/Images/ahasan1001.jpg  
  inflating: data/obj/Images/ahasan1001.txt  
  inflating: data/obj/Images/ahasan1003.jpg  
  inflating: data/obj/Images/ahasan1003.txt  
  inflating: data/obj/Images/ahasan1004.jpg  
  inflating: data/obj/Images/ahasan1004.txt  
  inflating: data/obj/Images/ahasan101.jpg  
  inflating: data/obj/Images/ahasan101.txt  
  inflating: data/obj/Images/ahasan1011.jpg  
  inflating: data/obj/Images/ahasan1011.txt  
  inflating: data/obj/Images/ahasan1024.jpg  
  inflating: data/obj/Images/ahasan1024.txt  
  inflating: data/obj/Images/ahasan1028.jpg  
  inflating: data/obj/Images/ahasan1028.txt  
  inflating: data/obj/Images/ahasan103.jpg  
  inflating:

**6) Create train.txt file**

In [ ]:
import glob
images_list = glob.glob("data/obj/Images/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

**7) Download pre-trained weights for the convolutional layers file**

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-09-10 15:15:09--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  50.5MB/s    in 3.1s    

2021-09-10 15:15:12 (50.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**8) Start training**

In [ ]:
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.695037), count: 4, class_loss = 0.073931, iou_loss = 0.298442, total_loss = 0.372373 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.682668), count: 2, class_loss = 0.224935, iou_loss = 0.159478, total_loss = 0.384413 
 total_bbox = 88308, rewritten_bbox = 0.103048 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.699738), count: 2, class_loss = 0.039030, iou_loss = 0.156503, total_loss = 0.195533 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.765341), count: 2, class_loss = 0.019956, iou_loss = 0.045716, total_loss = 0.065672 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.491442), count: 1, class_loss = 0.088502, iou_loss = 0.052208, total_loss = 0.140710 
 total_bbox = 88313, rewritten_bbox = 0.103043 % 
v3 (mse